In [ ]:
import requests
import csv
import datetime
from tabulate import tabulate
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


## Function api_calls - fetches data from the API

def api_calls():

    ## Get the exchanges data
    url = "https://www.cryptingup.com/api/exchanges"
    exchg = requests.get(url)
    exchg_list = dict(exchg.json())
    exchg_ig = exchg_list['exchanges']
    exchg_final_list = []
    asst_list = []
    for i in exchg_ig:
        exchn_name = i['id']
        exchn_vol = i['volume_24h']
    ## Get the Market data with exchane id
        url = "https://www.cryptingup.com/api/exchanges/{}/markets".format(i['id'])
        mrkt = requests.get(url)
        mrkt_list = dict(mrkt.json())
        mrkt_ig = mrkt_list['markets']

        for j in mrkt_ig:
            asst_exchange = j['exchange_id']
            asst_name = j['base_asset']
            asst_price = j['price']
            asst_vol_24h = j['volume_24h']
            asst_symbol = j['symbol']
            asst_change_24h = j['change_24h']
            asst_list.append([asst_exchange,asst_symbol,asst_name,asst_price,asst_vol_24h,asst_change_24h])

    fields = ['Exchg','symbol','asset','price','vol24','change24']
    with open(filename, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)
        csvwriter.writerows(asst_list)
        csvfile.close()

        #print(tabulate(asst_list,headers = ["Exchange Id","Asset","Price","24H Volume"]))
def plot_values():
    #print(os.getcwd())
    
    stock_df = pd.read_csv(filename)
    print(" TOP ASSET BASED ON PRICE ACROSS ALL EXCHANGES")
    stock_val = stock_df.sort_values(by = 'price', ascending = False)[:30]
    display(stock_val)
    print(" \n")
    print("AVERAGE PRICE OF ASSETS IN EACH EXCHANGE")
    display(stock_df.groupby(by = 'Exchg')['price'].mean())
    print(" \n")
    print("TOP ASSETS OF EXCHANGE BY PRICE")
    coin_df = stock_df[stock_df['Exchg'].isin(['COINBASE'])]
    sort_coin = coin_df.sort_values(by = 'price', ascending = False)[:5]
    display(sort_coin)
    print(" \n")
    print("TOP ASSETS OF EXCHANGE BY 24H VOLUME")
    coin_df = stock_df[stock_df['Exchg'].isin(['COINBASE'])]
    sort_coin1 = coin_df.sort_values(by = 'vol24', ascending = False)[:5]
    display(sort_coin1)
    
    plt.figure(figsize = (12,5))
    plt.bar(stock_df['Exchg'], stock_df['change24'])
    plt.title('24h changes in Exchanges')
    plt.xlabel('Exchange Names')
    plt.ylabel('24h change');
   

    plt.figure(figsize=(25,12))
    plt.hist(stock_df['Exchg'], color = 'red')
    plt.title("24h Volume");



filename = "./exch_asset"+(str(datetime.datetime.now()).replace(" ",""))+'.csv'
api_calls()
plot_values()
